# AMR Data Wrangling and Standardization

This notebook performs comprehensive data wrangling on the AMR (Antimicrobial Resistance) dataset, focusing on standardizing values and enriching the data with additional information.

## 🎯 Objectives
- Standardize DEPARTMENT codes ('Out'/'Inp' → 'Out-patient'/'In-patient')
- Map organism codes to human-readable names
- Add organism type classifications
- Standardize SEX codes ('f'/'m' → 'Female'/'Male')
- Maintain data integrity throughout transformations

## 📋 Data Flow
1. Load the dataset and reference tables
2. Analyze current data state
3. Apply standardizations and mappings
4. Validate transformations
5. Export enriched dataset

In [25]:
# Comprehensive AMR Data Wrangling Workflow

This notebook performs comprehensive data wrangling on the AMR dataset, including:

## 🎯 **Objectives:**
- Map organism codes to human-readable organism names
- Add organism type classifications (Gram-positive, Gram-negative, etc.)
- Standardize SEX codes ('f'/'m' → 'Female'/'Male')
- Standardize DEPARTMENT codes ('Out'/'Inp' → 'Out-patient'/'In-patient')
- Maintain data integrity throughout all transformations
- Provide comprehensive validation and quality checks

## 📋 **Data Flow:**
1. **Data Import**: Load cleaned dataset and reference tables
2. **Data Exploration**: Examine structure and content
3. **Organism Name Mapping**: Map codes to full organism names
4. **Organism Type Mapping**: Add organism classifications
5. **Sex Code Standardization**: Standardize sex codes to full labels
6. **Department Code Standardization**: Standardize department codes
7. **Final Export**: Save fully enriched dataset
8. **Validation**: Comprehensive quality checks and summary

---

# Load the latest dataset with standardized SEX codes
import pandas as pd
import os
from datetime import datetime

print("📂 LOADING LATEST DATASET FOR DEPARTMENT STANDARDIZATION")
print("=" * 60)

# Load the most recent dataset
input_file = r'c:\NATIONAL AMR DATA ANALYSIS FILES\data\processed\mapped\df_final_with_standardized_sex_2025-06-12_15-32-27.csv'
df = pd.read_csv(input_file)

print(f"✅ Dataset loaded successfully!")
print(f"   📊 Shape: {df.shape}")
print(f"   📁 Source: {input_file}")
print(f"   📋 Columns: {len(df.columns)}")

📂 LOADING LATEST DATASET FOR DEPARTMENT STANDARDIZATION
✅ Dataset loaded successfully!
   📊 Shape: (32688, 47)
   📁 Source: c:\NATIONAL AMR DATA ANALYSIS FILES\data\processed\mapped\df_final_with_standardized_sex_2025-06-12_15-32-27.csv
   📋 Columns: 47


c:\Users\MAdu\AppData\Local\anaconda3\envs\venv\lib\site-packages\IPython\core\interactiveshell.py:3508: DtypeWarning: Columns (27,29,32,33,34,37,41,44) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [30]:
# 1. DATA IMPORT AND SETUP
# Import required libraries and load the cleaned dataset along with reference tables

import pandas as pd
import os
from datetime import datetime

print("📂 LOADING DATA FILES...")
print("=" * 50)

# Load the cleaned dataset and reference tables
df_cleaned = pd.read_csv(r'C:\NATIONAL AMR DATA ANALYSIS FILES\data\processed\deduplicated\df_cleaned_2025-06-11.csv')
df_organism_ref = pd.read_csv(r'C:\NATIONAL AMR DATA ANALYSIS FILES\data\Database Resources\Organisms_Data_Final.csv')
df_antimicrobial_ref = pd.read_csv(r'C:\NATIONAL AMR DATA ANALYSIS FILES\data\Database Resources\Antimicrobials_Data_Final.csv')

print("✅ Data files loaded successfully!")
print(f"📊 Main dataset shape: {df_cleaned.shape}")
print(f"🦠 Organism reference table: {df_organism_ref.shape}")
print(f"💊 Antimicrobial reference table: {df_antimicrobial_ref.shape}")

# Create working copy
df_final = df_cleaned.copy()
print(f"\n🔧 Working dataframe 'df_final' created with shape: {df_final.shape}")

# Analyze DEPARTMENT column
print("\n🏥 DEPARTMENT COLUMN ANALYSIS")
print("=" * 40)

if 'DEPARTMENT' in df.columns:
    print(f"✅ DEPARTMENT column found!")
    
    # Check current values
    print(f"\n📊 Current DEPARTMENT values (before standardization):")
    dept_counts = df['DEPARTMENT'].value_counts(dropna=False)
    total_records = len(df)
    
    for value, count in dept_counts.items():
        percentage = (count / total_records) * 100
        if pd.notna(value):
            print(f"   '{value}' (length: {len(str(value))}): {count:,} records ({percentage:.1f}%)")
        else:
            print(f"   Missing/NaN: {count:,} records ({percentage:.1f}%)")
    
    print(f"\n📈 Summary:")
    print(f"   Total records: {total_records:,}")
    print(f"   Records with DEPARTMENT data: {df['DEPARTMENT'].notna().sum():,}")
    print(f"   Records missing DEPARTMENT data: {df['DEPARTMENT'].isna().sum():,}")
    
    # Check for leading/trailing spaces
    if df['DEPARTMENT'].notna().any():
        values_with_spaces = df['DEPARTMENT'].dropna().apply(lambda x: str(x) != str(x).strip()).sum()
        print(f"   Values with leading/trailing spaces: {values_with_spaces:,}")
    
    # Show sample data
    print(f"\n🔍 Sample DEPARTMENT values:")
    sample_data = df[df['DEPARTMENT'].notna()][['ORGANISM_NAME', 'DEPARTMENT']].head(10)
    for idx, row in sample_data.iterrows():
        organism_name = str(row['ORGANISM_NAME'])[:25] if pd.notna(row['ORGANISM_NAME']) else 'Unknown'
        print(f"   {organism_name:25} | DEPT: '{row['DEPARTMENT']}'")
        
else:
    print(f"❌ DEPARTMENT column not found!")
    print(f"Available columns containing 'dept':")
    dept_cols = [col for col in df.columns if 'dept' in col.lower()]
    if dept_cols:
        for col in dept_cols:
            print(f"   - {col}")
    else:
        print(f"   No columns found containing 'dept'")
        print(f"\n📋 All columns:")
        for i, col in enumerate(df.columns, 1):
            print(f"   {i:2d}. {col}")

📂 LOADING DATA FILES...
✅ Data files loaded successfully!
📊 Main dataset shape: (32688, 45)
🦠 Organism reference table: (2946, 7)
💊 Antimicrobial reference table: (392, 5)

🔧 Working dataframe 'df_final' created with shape: (32688, 45)

🏥 DEPARTMENT COLUMN ANALYSIS
✅ DEPARTMENT column found!

📊 Current DEPARTMENT values (before standardization):
   'Out-patient' (length: 11): 16,858 records (51.6%)
   'In-patient' (length: 10): 15,830 records (48.4%)

📈 Summary:
   Total records: 32,688
   Records with DEPARTMENT data: 32,688
   Records missing DEPARTMENT data: 0
   Values with leading/trailing spaces: 0

🔍 Sample DEPARTMENT values:
   No growth                 | DEPT: 'Out-patient'
   No growth                 | DEPT: 'Out-patient'
   No growth                 | DEPT: 'In-patient'
   No growth                 | DEPT: 'In-patient'
   No growth                 | DEPT: 'Out-patient'
   No growth                 | DEPT: 'Out-patient'
   No growth                 | DEPT: 'In-patient'
   No

In [31]:
# 2. DATA EXPLORATION AND STRUCTURE ANALYSIS
# Examine the structure of both the main dataset and reference tables

print("🔍 MAIN DATASET ANALYSIS")
print("=" * 50)
print(f"Shape: {df_final.shape}")
print(f"\nFirst few ORGANISM_CODE values:")
print(df_final['ORGANISM_CODE'].head(10).tolist())
print(f"\nUnique organism codes (first 20):")
print(df_final['ORGANISM_CODE'].unique()[:20].tolist())
print(f"\nTotal unique organism codes: {df_final['ORGANISM_CODE'].nunique()}")

print("\n🧬 ORGANISM REFERENCE TABLE ANALYSIS")
print("=" * 50)
print(f"Shape: {df_organism_ref.shape}")
print(f"\nReference table columns:")
print(df_organism_ref.columns.tolist())
print(f"\nSample organism mappings:")
print(df_organism_ref[['ORGANISM_CODE', 'ORGANISM_NAME', 'ORGANISM_TYPE_DESCRIPTION']].head(10))
print(f"\nUnique organism codes in reference: {df_organism_ref['ORGANISM_CODE'].nunique()}")
print(f"Available organism types: {df_organism_ref['ORGANISM_TYPE_DESCRIPTION'].unique()}")

# Check key columns existence
print(f"\n🔍 KEY COLUMNS CHECK:")
key_columns = ['ORGANISM_CODE', 'SEX', 'DEPARTMENT']
for col in key_columns:
    exists = col in df_final.columns
    print(f"   {col}: {'✅ Found' if exists else '❌ Missing'}")
    if exists and col in ['SEX', 'DEPARTMENT']:
        unique_vals = df_final[col].unique()[:10]
        print(f"      Sample values: {unique_vals}")

🔍 MAIN DATASET ANALYSIS
Shape: (32688, 45)

First few ORGANISM_CODE values:
['xxx', 'xxx', 'xxx', 'xxx', 'xxx', 'xxx', 'xxx', 'xxx', 'xxx', 'xxx']

Unique organism codes (first 20):
['xxx', 'eco', 'ci-', 'kl-', 'scn', 'sep', 'sal', 'sau', 'ac-', 'sta', 'en-', 'pae', 'ent', 'kpn', 'str', 'pr-', 'ste', 'ps-', 'pmi', 'sat']

Total unique organism codes: 76

🧬 ORGANISM REFERENCE TABLE ANALYSIS
Shape: (2946, 7)

Reference table columns:
['ORGANISM_CODE', 'ORGANISM_NAME', 'ORGANISM_TYPE', 'ORGANISM_TYPE_DESCRIPTION', 'IS_COMMON', 'EXTRACTION_DATE', 'DATA_SOURCE']

Sample organism mappings:
  ORGANISM_CODE                           ORGANISM_NAME  \
0           NaN                           Nannizzia sp.   
1           103                   Escherichia coli O103   
2           104           Salmonella Typhimurium DT 104   
3           111                   Escherichia coli O111   
4           135  Neisseria meningitidis, serogroup W135   
5           139                    Vibrio cholerae O139

In [32]:
# Export the dataset with standardized DEPARTMENT column
print("\n💾 EXPORTING STANDARDIZED DATASET")
print("=" * 40)

# Create output file path
output_file = r'c:\NATIONAL AMR DATA ANALYSIS FILES\Data_Department_Standardized.csv'

# Save the dataset
df.to_csv(output_file, index=False)
file_size_mb = os.path.getsize(output_file) / (1024*1024)

print(f"✅ Dataset exported successfully!")
print(f"   📁 Location: {output_file}")
print(f"   📏 File size: {file_size_mb:.2f} MB")
print(f"   📊 Records: {len(df):,}")
print(f"   📋 Columns: {len(df.columns)}")

# Final summary
print(f"\n🎯 FINAL SUMMARY:")
if 'DEPARTMENT' in df.columns:
    dept_dist = df['DEPARTMENT'].value_counts(dropna=False)
    print(f"   📊 DEPARTMENT distribution:")
    for value, count in dept_dist.items():
        percentage = (count / len(df)) * 100
        if pd.notna(value):
            print(f"      '{value}': {count:,} records ({percentage:.1f}%)")
        else:
            print(f"      Missing/NaN: {count:,} records ({percentage:.1f}%)")
else:
    print(f"   ⚠️ DEPARTMENT column not found in dataset")

print(f"\n🎉 DEPARTMENT STANDARDIZATION COMPLETED!")
print(f"📄 Final file: Data_Department_Standardized.csv")
print(f"📈 Transformations applied:")
print(f"   • 'Out' → 'Out-patient'")
print(f"   • 'Inp' → 'In-patient'")
print(f"   • Leading/trailing spaces removed")
print(f"   • Case-insensitive matching")

# 3. ORGANISM NAME MAPPING
# Map organism codes to their corresponding organism names using case-insensitive matching

print("🔧 PREPARING DATA FOR ORGANISM NAME MAPPING")
print("=" * 60)

# Clean and normalize organism codes for case-insensitive matching
df_final['ORGANISM_CODE_CLEAN'] = df_final['ORGANISM_CODE'].astype(str).str.strip().str.lower()
df_organism_ref['ORGANISM_CODE_CLEAN'] = df_organism_ref['ORGANISM_CODE'].astype(str).str.strip().str.lower()

print("Sample of cleaned organism codes from main dataset:")
print(df_final['ORGANISM_CODE_CLEAN'].head(10).tolist())
print("\nSample of cleaned organism codes from reference:")
print(df_organism_ref['ORGANISM_CODE_CLEAN'].head(10).tolist())

# Create a mapping dictionary from the reference table
organism_mapping = df_organism_ref.drop_duplicates(subset=['ORGANISM_CODE_CLEAN']).set_index('ORGANISM_CODE_CLEAN')['ORGANISM_NAME'].to_dict()

print(f"\n📋 MAPPING DICTIONARY CREATED")
print(f"Total unique organism codes in reference: {len(organism_mapping)}")
print("Sample mapping entries:")
for i, (code, name) in enumerate(list(organism_mapping.items())[:5]):
    print(f"  {code} → {name}")

print(f"\n🎯 APPLYING ORGANISM NAME MAPPING")
print("=" * 50)

# Perform the mapping
df_final['ORGANISM_NAME'] = df_final['ORGANISM_CODE_CLEAN'].map(organism_mapping)

# Check mapping results
total_records = len(df_final)
successfully_mapped = df_final['ORGANISM_NAME'].notna().sum()
unmapped_count = df_final['ORGANISM_NAME'].isna().sum()

print(f"Mapping Results:")
print(f"  📊 Total records: {total_records:,}")
print(f"  ✅ Successfully mapped: {successfully_mapped:,}")
print(f"  ❌ Unmapped records: {unmapped_count:,}")
print(f"  📈 Success rate: {(successfully_mapped / total_records) * 100:.2f}%")

# Reposition ORGANISM_NAME column next to ORGANISM_CODE
organism_code_idx = df_final.columns.get_loc('ORGANISM_CODE')
columns = list(df_final.columns)
columns.remove('ORGANISM_NAME')
columns.remove('ORGANISM_CODE_CLEAN')
columns.insert(organism_code_idx + 1, 'ORGANISM_NAME')
df_final = df_final[columns].copy()

print(f"\n✅ ORGANISM_NAME positioned next to ORGANISM_CODE")
print(f"📊 Most common organisms in dataset:")
organism_counts = df_final['ORGANISM_NAME'].value_counts().head(5)
for organism, count in organism_counts.items():
    percentage = (count / len(df_final)) * 100
    print(f"  {organism}: {count:,} records ({percentage:.1f}%)")


💾 EXPORTING STANDARDIZED DATASET
✅ Dataset exported successfully!
   📁 Location: c:\NATIONAL AMR DATA ANALYSIS FILES\Data_Department_Standardized.csv
   📏 File size: 4.20 MB
   📊 Records: 32,688
   📋 Columns: 47

🎯 FINAL SUMMARY:
   📊 DEPARTMENT distribution:
      'Out-patient': 16,858 records (51.6%)
      'In-patient': 15,830 records (48.4%)

🎉 DEPARTMENT STANDARDIZATION COMPLETED!
📄 Final file: Data_Department_Standardized.csv
📈 Transformations applied:
   • 'Out' → 'Out-patient'
   • 'Inp' → 'In-patient'
   • Leading/trailing spaces removed
   • Case-insensitive matching
🔧 PREPARING DATA FOR ORGANISM NAME MAPPING
Sample of cleaned organism codes from main dataset:
['xxx', 'xxx', 'xxx', 'xxx', 'xxx', 'xxx', 'xxx', 'xxx', 'xxx', 'xxx']

Sample of cleaned organism codes from reference:
['nan', '103', '104', '111', '135', '139', '145', '149', '157', '1k1']

📋 MAPPING DICTIONARY CREATED
Total unique organism codes in reference: 2353
Sample mapping entries:
  nan → Nannizzia sp.
  103 

# Department Column Standardization

This section standardizes the DEPARTMENT column values in the AMR dataset for better consistency and readability.

## Transformations Applied:
- **"Out"** (case-insensitive) → **"Out-patient"**
- **"Inp"** (case-insensitive) → **"In-patient"**
- **Space cleaning**: Remove leading and trailing spaces
- **Case handling**: Support uppercase, lowercase, and mixed case inputs

## Process:
1. Load the latest dataset with standardized SEX codes
2. Analyze current DEPARTMENT column values
3. Clean spaces and apply standardization mapping
4. Validate transformation results
5. Export as `Data_Department_Standardized.csv`

# 4. ORGANISM TYPE MAPPING
# Map organism codes to their corresponding organism types (classifications)

print("🔬 APPLYING ORGANISM TYPE MAPPING")
print("=" * 50)

# Create mapping dictionary for organism types from reference table
organism_type_mapping = df_organism_ref.drop_duplicates(subset=['ORGANISM_CODE_CLEAN']).set_index('ORGANISM_CODE_CLEAN')['ORGANISM_TYPE_DESCRIPTION'].to_dict()

print(f"📋 Organism type mapping dictionary created")
print(f"  Total unique codes for type mapping: {len(organism_type_mapping)}")
print(f"  Sample organism type mappings:")
for i, (code, type_desc) in enumerate(list(organism_type_mapping.items())[:5]):
    print(f"    {code} → {type_desc}")

# Apply organism type mapping
df_final['ORGANISM_CODE_CLEAN_TEMP'] = df_final['ORGANISM_CODE'].astype(str).str.strip().str.lower()
df_final['ORGANISM_TYPE'] = df_final['ORGANISM_CODE_CLEAN_TEMP'].map(organism_type_mapping)
df_final = df_final.drop('ORGANISM_CODE_CLEAN_TEMP', axis=1)

# Reposition ORGANISM_TYPE column next to ORGANISM_NAME
organism_name_idx = df_final.columns.get_loc('ORGANISM_NAME')
columns = list(df_final.columns)
columns.remove('ORGANISM_TYPE')
columns.insert(organism_name_idx + 1, 'ORGANISM_TYPE')
df_final = df_final[columns].copy()

# Check mapping results
total_records = len(df_final)
types_mapped = df_final['ORGANISM_TYPE'].notna().sum()
type_success_rate = (types_mapped / total_records) * 100

print(f"\n📊 Organism type mapping results:")
print(f"  Total records: {total_records:,}")
print(f"  Successfully mapped types: {types_mapped:,}")
print(f"  Type mapping success rate: {type_success_rate:.2f}%")

print(f"\n🔬 ORGANISM TYPE DISTRIBUTION:")
type_counts = df_final['ORGANISM_TYPE'].value_counts()
for organism_type, count in type_counts.items():
    percentage = (count / len(df_final)) * 100
    print(f"  {organism_type}: {count:,} records ({percentage:.1f}%)")

In [ ]:
print("💾 EXPORTING STANDARDIZED DATASET")
print("=" * 40)

# Create output file path
output_file = r'c:\NATIONAL AMR DATA ANALYSIS FILES\Data_Department_Standardized.csv'

# Save the dataset
df.to_csv(output_file, index=False)
file_size_mb = os.path.getsize(output_file) / (1024*1024)

print(f"✅ Dataset exported successfully!")
print(f"   📁 Location: {output_file}")
print(f"   📏 File size: {file_size_mb:.2f} MB")
print(f"   📊 Records: {len(df):,}")
print(f"   📋 Columns: {len(df.columns)}")

print(f"\n🎉 DEPARTMENT STANDARDIZATION COMPLETED!")
print(f"📄 Final file: Data_Department_Standardized.csv")
print(f"📈 Transformations applied:")
print(f"   • Case-insensitive mapping (Out → Out-patient, Inp → In-patient)")
print(f"   • Leading/trailing space removal")
print(f"   • NaN value handling")